In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from cuml.ensemble import RandomForestClassifier
from cuml import LogisticRegression
from sklearn.ensemble import VotingClassifier
from cuml.common.device_selection import set_global_device_type, get_global_device_type
from model.tuning import *
from dataclasses import dataclass, asdict
from itertools import product
from tqdm import tqdm

set_global_device_type('gpu')
print('new device type:', get_global_device_type())

new device type: DeviceType.device


In [2]:
def ensemble_classifier(max_depth, n_estimators, C, l1_ratio, penalty, rfc_weight, n_bins: int = 128):
    # define classifiers
    rfc = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, n_bins=n_bins)
    lr = LogisticRegression(C=C, l1_ratio=l1_ratio, penalty=penalty)
    
    # define voting ensemble
    voting_model = VotingClassifier(
        estimators=[
            ('rfc', rfc), 
            ('lr', lr),
        ],
        voting='soft',
        weights=[rfc_weight, 1.0-rfc_weight],
    )
    return voting_model

@dataclass
class AblationEvalObj:
    token_length: bool
    patient_demographics: bool
    mds_updrs: bool
    moca: bool
    acc: float
    micro_f1: float
    macro_f1: float
    dict = asdict

In [3]:
# define model to optimize
model_function = ensemble_classifier
model_name = "voting_ensemble"

# define hyperparameter ranges to search
# we aren't optimizing hyperparams so we'll leave this blank
hparam_range_dict = {}

# define optimizer
hparam_optimizer = HyperParamOptimizer(
    model_function=model_function, 
    model_name=model_name, 
    hparam_range_dict=hparam_range_dict
)

In [4]:
# load data into optimizer
train_val_path = "./data/fallreports_2023-9-21_train.csv"
test_path = "./data/fallreports_2023-9-21_test.csv"
hparam_optimizer.load_data(
    train_val_path=train_val_path, 
    test_path=test_path
)

Data loaded...


In [5]:
# set model parameters
ngram_list = [1, 2]
target_feature = 'fog_q_class'
vector_type = 'tf-idf'
n_features = 250
seed = 13
k = 5
constant_dict = {}

In [6]:
ensemble_best_param_dict = {
    'C': 2045.7891312130546,
    'l1_ratio': 0.049833394439413,
    'max_depth': 48.12718850106587,
    'n_estimators': 97.06033604146855,
    'penalty': 'elasticnet',
    'rfc_weight': 0.855137554079295
}

# set sample size to -1 to bootstrap results
sample_size = -1

n_replications = 3

augment_cat_list = ['token_length', 'patient_demographics', 'mds_updrs', 'moca']
level_list = [True, False]

eval_list = []

# iterate over all factor combinations and repeat all n_replications times
for curr_level_list in tqdm(list(product(level_list, repeat=len(augment_cat_list)))*n_replications):
    # build current augmentation dict
    curr_aug_dict = dict(zip(augment_cat_list, curr_level_list))
    
    print(f"\nAugmentations: {curr_aug_dict}")
    
    # update model params
    hparam_optimizer.set_params(
        ngram_list=ngram_list, 
        target_feature=target_feature, 
        vector_type=vector_type, 
        n_features=n_features, 
        augment_dict=curr_aug_dict, 
        seed=seed,
        k=k,
        constant_dict=constant_dict
    )
    
    # train and evaluate model
    acc, micro_f1, macro_f1 = hparam_optimizer.objective_function(
        ensemble_best_param_dict, 
        sample_size=sample_size, 
        log=False, 
        return_all=True,
        rfc=True
    )
    eval_list.append(
        AblationEvalObj(
            token_length=curr_aug_dict['token_length'],
            patient_demographics=curr_aug_dict['token_length'],
            mds_updrs=curr_aug_dict['mds_updrs'],
            moca=curr_aug_dict['moca'], 
            acc=acc, 
            micro_f1=micro_f1, 
            macro_f1=macro_f1
        )
    )

  0%|          | 0/48 [00:00<?, ?it/s]


Augmentations: {'token_length': True, 'patient_demographics': True, 'mds_updrs': True, 'moca': True}
Parameters updated...
Randomly sampling 5 sets with size -1 from train set...
[W] [15:37:53.015226] QWL-QN: max iterations reached
[W] [15:37:53.016017] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [15:37:57.727871] QWL-QN: max iterations reached
[W] [15:37:57.728052] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [15:38:00.910103] QWL-QN: max iterations reached
[W] [15:38:00.910200] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [15:38:05.645536] QWL-QN: max iterations reached
[W] [15:38:

  2%|▏         | 1/48 [00:27<21:15, 27.14s/it]

[W] [15:38:08.817514] QWL-QN: max iterations reached
[W] [15:38:08.817688] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.

Accuracy: 0.8676
Micro-Averaged F1: 0.8646
Macro-Averaged F1: 0.8639

Augmentations: {'token_length': True, 'patient_demographics': True, 'mds_updrs': True, 'moca': False}
Parameters updated...
Randomly sampling 5 sets with size -1 from train set...
[W] [15:38:13.500099] QWL-QN: max iterations reached
[W] [15:38:13.500201] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [15:38:16.814978] QWL-QN: max iterations reached
[W] [15:38:16.815122] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input

  4%|▍         | 2/48 [00:44<16:28, 21.48s/it]

...


Accuracy: 0.7127
Micro-Averaged F1: 0.7052
Macro-Averaged F1: 0.7054

Augmentations: {'token_length': False, 'patient_demographics': False, 'mds_updrs': False, 'moca': True}
Parameters updated...
Randomly sampling 5 sets with size -1 from train set...


 98%|█████████▊| 47/48 [13:40<00:16, 16.12s/it]


Accuracy: 0.8225
Micro-Averaged F1: 0.8195
Macro-Averaged F1: 0.8191

Augmentations: {'token_length': False, 'patient_demographics': False, 'mds_updrs': False, 'moca': False}
Parameters updated...
Randomly sampling 5 sets with size -1 from train set...


100%|██████████| 48/48 [13:54<00:00, 17.38s/it]


Accuracy: 0.6648
Micro-Averaged F1: 0.6448
Macro-Averaged F1: 0.6444


In [7]:
eval_df = pd.DataFrame([eval_obj.dict() for eval_obj in eval_list])
eval_df

,token_length,patient_demographics,mds_updrs,moca,acc,micro_f1,macro_f1
0,True,True,True,True,0.867606,0.864553,0.863915
1,True,True,True,False,0.845070,0.843304,0.842623
2,True,True,False,True,0.898592,0.900552,0.900593
3,True,True,False,False,0.814085,0.810344,0.809766
4,True,True,True,True,0.808451,0.802325,0.802504
5,True,True,True,False,0.676056,0.676056,0.674984
6,True,True,False,True,0.811268,0.803519,0.803505
7,True,True,False,False,0.653521,0.645533,0.645528
8,False,False,True,True,0.867606,0.865329,0.865225
9,False,False,True,False,0.847887,0.845714,0.845444


In [8]:
eval_df.to_csv("./ablation_study_results.csv")

In [15]:
len(list(product(level_list, repeat=len(augment_cat_list)))*3)

48